In [1]:
import os
import numpy as np

In [2]:
from paths import rootdir as datadir
from prepare import param_generator

In [3]:
import graphs

<IPython.core.display.Javascript object>

In [4]:
n_trials, cue_freqs, rew_freqs, params = param_generator()
exps = {(n_trial, cue_freq, rew_freq): idx for idx, n_trial, cue_freq, rew_freq in params}

In [5]:
def load_records(n_trial, cue_freq, rew_freq):
    """Load the data for a given combination of parameters."""
    idx = exps[(n_trial, cue_freq, rew_freq)]
    filename = 'data_sgstim.{}'.format(idx)
    filepath = os.path.join(datadir, 'data', filename + '.npy')
    return np.squeeze(np.load(filepath))

### Best Choice graph

Display the percentage over 100 repetitions of the best cue being selected at a given trial. The percentage is maximum during the first trials because only one stimulus is presented.

In [6]:
from bokeh.io import push_notebook

def value_best(n_trial, cue_freq, rew_freq, figs=(None, None), lines=(None, None), handle=None, show=True):
    """Display graph of best choice"""
    records = load_records(n_trial, cue_freq, rew_freq)
    _, fig_best,  lines_best  = graphs.best(records, fig=figs[0], lines=lines[0], handle=handle, show=False)
    _, fig_value, lines_value = graphs.value(records, fig=figs[1], lines=lines[1], handle=handle, show=False)
    figs, lines = [fig_best, fig_value], [lines_best, lines_value]
    fig_best.title.text = "Cue choice (best) [n_trial={}, A_freq={}, A_rew={}]".format(n_trial, cue_freq, rew_freq)
    fig_value.title.text = "Stimulus value [n_trial={}, A_freq={}, A_rew={}]".format(n_trial, cue_freq, rew_freq)
    if show:
        handle = graphs.show(graphs.column(fig_best, fig_value), notebook_handle=True)
    return handle, figs, lines
    
def update_vb(n_trial, cue_freq, rew_freq, figs=(None, None), lines=(None, None), handle=None):
    value_best(n_trial, cue_freq, rew_freq, figs=figs, lines=lines, handle=handle, show=False)
    push_notebook(handle=handle)

In [7]:
idx, n_trial, cue_freq, rew_freq = params[0]
handle_best, figs, lines = value_best(n_trial, cue_freq, rew_freq)

In [8]:
graphs.interact(update_vb, figs=graphs.fixed(figs), lines=graphs.fixed(lines), handle=graphs.fixed(handle_best),
    n_trial=graphs.select('n_trial', n_trials), cue_freq=graphs.select('A_freq', cue_freqs), 
    rew_freq=graphs.select('A_rew', rew_freqs))

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


### Chosen Cue Graph

In [9]:
def phases(n_trial):
    data = {}

    x, y, size, colors = [], [], [], []

    for cue_freq in cue_freqs:
        for rew_freq in rew_freqs:
            records = load_records(n_trial, cue_freq, rew_freq)
            cues = records["cue"]
            cue_table = graphs.cue_count(cues)
            A_freq = np.mean(cue_table[0][n_trial:n_trial+10])
            B_freq = np.mean(cue_table[1][n_trial:n_trial+10])
    
            x.append(cue_freq)
            y.append(rew_freq)
            if A_freq >= B_freq:
                size.append(np.sqrt(A_freq-B_freq)/25)
                colors.append("#fa6900")
            else:
                size.append(np.sqrt(B_freq-A_freq)/25)
                colors.append("#69d2e7")
                
    fig = graphs.figure(x_range=[-0.1, 1.1], y_range=[-0.1, 1.1],
                        plot_width=500, plot_height=500, tools="")
            
    fig.xaxis.axis_label = "A frequency"
    fig.yaxis.axis_label = "A reward"
        
    fig.rect(x, y, size, size, color=colors)
    graphs.show(fig)
    
phases(0)

In [10]:
phases(10)

In [11]:
phases(20)

In [12]:
phases(100)

In [13]:
def graph_cue(n_trial, cue_freq, rew_freq, fig=None, lines=None, handle=None, show=True):
    """Display graph of best choice"""
    records = load_records(n_trial, cue_freq, rew_freq)
    handle, fig, lines = graphs.cue(records, fig=fig, lines=lines, handle=handle, show=show)
    fig.title.text = "Cue choice [n_trial={}, A_freq={}, A_rew={}]".format(n_trial, cue_freq, rew_freq)
    return handle, fig, lines

def update_cue(n_trial, cue_freq, rew_freq, fig=None, lines=None, handle=None):
    graph_cue(n_trial, cue_freq, rew_freq, fig=fig, lines=lines, show=False)

In [14]:
idx, n_trial, cue_freq, rew_freq = params[0]
handle_cue, fig, lines = graph_cue(n_trial, cue_freq, rew_freq)

In [15]:
graphs.interact(update_cue, fig=graphs.fixed(fig), lines=graphs.fixed(lines), handle=graphs.fixed(handle_cue),
    n_trial=graphs.select('number of trials for single phase', n_trials), cue_freq=graphs.select('A freq. during single phase', cue_freqs), 
    rew_freq=graphs.select('A reward', rew_freqs), single_phase=True)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
